<a href="https://colab.research.google.com/github/LeeRinji/Replication-Occupational-Networks/blob/main/Read_Data_CPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Packages and Data**

In [1]:
import requests
import csv
import numpy
import re
import time
import random
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Read Recent CSV Format Data**

In [3]:
df_test=pd.read_csv("/content/gdrive/MyDrive/Avail_Data/pppub22.csv")

In [4]:
df_test

,PERIDNUM,PH_SEQ,P_SEQ,A_LINENO,PF_SEQ,PHF_SEQ,OED_TYP1,OED_TYP2,OED_TYP3,PERRP,...,I_DISVL1,I_DISVL2,I_SURVL1,I_SURVL2,MIG_CBST,MIG_DSCP,DEP_STAT,FILEDATE,FILESTAT,YYYYMM
0,2022093493100051201101,1,1,1,1,1,0,0,0,41,...,0,0,0,0,0,0,0,72522,5,202203
1,2100059324402431401101,5,1,1,1,1,0,0,0,40,...,0,0,0,0,0,0,0,72522,1,202203
2,2100059324402431401102,5,2,2,1,1,0,0,0,42,...,0,0,0,0,0,0,0,72522,1,202203
3,1410320300869091411101,6,1,1,1,1,0,0,0,40,...,0,0,0,0,0,0,0,72522,1,202203
4,1410320300869091411102,6,2,2,1,1,0,0,0,42,...,0,0,0,0,0,0,0,72522,1,202203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152727,1102030217034551411102,89197,2,2,1,1,0,0,0,42,...,0,0,0,0,0,0,0,72522,1,202203
152728,1102030217034551411103,89197,3,3,1,1,0,0,0,48,...,0,0,0,0,0,0,1,72522,6,202203
152729,1102030217034551411104,89197,4,4,1,1,0,0,0,48,...,0,0,0,0,0,0,1,72522,6,202203
152730,1102030217034551411105,89197,5,5,1,1,0,0,0,48,...,0,0,0,0,0,0,1,72522,6,202203


In [6]:
col_names=df_test.columns.to_list()

# **Read Previous ASCII Data File**

In [10]:
with open("/content/gdrive/MyDrive/Avail_Data/mar98pub.cps",'r') as f:
  df_raw_98=f.read()

In [16]:
df_raw_98[:1000]

'163569002070000000010101703811010021011110064000000000221400020202000000022200000000200001003050020000000200000002000000020000000200000002000000200000020000000200000002000000020000000200000002000000020000000200000002000000020000000200000002000000000030500000305000000000013222120020900000093140000000000000000000000000000000000000000000000000726994491808233900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000\n2635690132010100000001010000000083504110000000100305002000000020000000200000002000000020000000200000020000002000000020000000200000002000000020000

In [17]:
records=df_raw_98.split('\n')

In [18]:
len(records)

253045

# **Data Transformation**

In [20]:
from collections import defaultdict
records_len_dict=defaultdict(int)
for item in records:
  records_len_dict[len(item)]+=1
print(records_len_dict)

defaultdict(<class 'int'>, {852: 253044, 0: 1})


In [21]:
records[-3]

'363068420200044710126430410801122202020100001010000113520000000000011153900000000000110000000000000000831315102000000000000000000000000000000000200021211016410523011000005230000001402200001831315011781152405401300140009090111610220001200000010220000000002000000000000200000020000000110210020000020000200000000200000000000200000000000000000000122000000000000000000002000000000000000000100975200000020000020000000002000002000002000000020000000025075000220000000030751114222111200000111110000200004000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000026801000000000000000001683000000000000000000000000002005000420504004005705705700100000000030000110000022200000002220020000020000000000000000000000040000000000000000000000000000000000020000000000000000'